In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,80.60,78,75,5.75,light rain,2021-02-01 03:39:33
1,1,Bengkulu,ID,-3.8004,102.2655,85.46,59,77,5.99,broken clouds,2021-02-01 03:39:39
2,2,Adrar,MR,20.5022,-10.0711,62.76,28,99,7.54,overcast clouds,2021-02-01 03:40:42
3,3,Tuktoyaktuk,CA,69.4541,-133.0374,-27.40,75,20,3.44,few clouds,2021-02-01 03:39:36
4,4,Monki,PL,53.4050,22.7979,12.99,88,20,7.96,few clouds,2021-02-01 03:40:42


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,80.60,78,75,5.75,light rain,2021-02-01 03:39:33
1,1,Bengkulu,ID,-3.8004,102.2655,85.46,59,77,5.99,broken clouds,2021-02-01 03:39:39
7,7,Kapaa,US,22.0752,-159.3190,75.20,69,40,10.36,scattered clouds,2021-02-01 03:38:49
8,8,Maputo,MZ,-25.9653,32.5892,78.80,89,75,6.91,broken clouds,2021-02-01 03:40:42
10,10,Butaritari,KI,3.0707,172.7902,81.25,82,53,14.03,broken clouds,2021-02-01 03:39:32
13,13,Canutama,BR,-6.5339,-64.3831,75.22,93,93,1.21,overcast clouds,2021-02-01 03:39:51
14,14,Sugod,PH,12.9870,124.0805,79.65,79,88,13.82,overcast clouds,2021-02-01 03:40:43
15,15,Nelson Bay,AU,-32.7167,152.1500,84.99,61,40,12.66,scattered clouds,2021-02-01 03:40:43
17,17,Busselton,AU,-33.6500,115.3333,86.68,29,0,6.11,clear sky,2021-02-01 03:39:33
19,19,Ixtapa,MX,20.7000,-105.2000,70.00,94,1,1.92,clear sky,2021-02-01 03:39:59


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Avarua,CK,-21.2078,-159.7750,80.60,78,75,5.75,light rain,2021-02-01 03:39:33
1,1,Bengkulu,ID,-3.8004,102.2655,85.46,59,77,5.99,broken clouds,2021-02-01 03:39:39
7,7,Kapaa,US,22.0752,-159.3190,75.20,69,40,10.36,scattered clouds,2021-02-01 03:38:49
8,8,Maputo,MZ,-25.9653,32.5892,78.80,89,75,6.91,broken clouds,2021-02-01 03:40:42
10,10,Butaritari,KI,3.0707,172.7902,81.25,82,53,14.03,broken clouds,2021-02-01 03:39:32
...,...,...,...,...,...,...,...,...,...,...,...
669,669,Singkawang,ID,0.9000,109.0000,83.23,78,47,6.78,light rain,2021-02-01 03:42:08
670,670,The Valley,AI,18.2170,-63.0578,78.80,73,20,13.80,few clouds,2021-02-01 03:39:58
672,672,High Rock,BS,26.6208,-78.2833,71.60,78,20,9.22,few clouds,2021-02-01 03:42:08
673,673,Matara,LK,5.9485,80.5353,81.75,72,52,4.97,broken clouds,2021-02-01 03:41:24


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,80.60,light rain,-21.2078,-159.7750,
1,Bengkulu,ID,85.46,broken clouds,-3.8004,102.2655,
7,Kapaa,US,75.20,scattered clouds,22.0752,-159.3190,
8,Maputo,MZ,78.80,broken clouds,-25.9653,32.5892,
10,Butaritari,KI,81.25,broken clouds,3.0707,172.7902,
13,Canutama,BR,75.22,overcast clouds,-6.5339,-64.3831,
14,Sugod,PH,79.65,overcast clouds,12.9870,124.0805,
15,Nelson Bay,AU,84.99,scattered clouds,-32.7167,152.1500,
17,Busselton,AU,86.68,clear sky,-33.6500,115.3333,
19,Ixtapa,MX,70.00,clear sky,20.7000,-105.2000,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
clean = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean.dropna()
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig